In [15]:
import pandas as pd
import yfinance as yf

wallet_antonio = pd.read_excel("antonio_ap.xlsx", sheet_name='base', header=0)
wallet_antonio.head(3)

,ticker,pm,plano
0,BHIA3.SA,0.200000,Vender
1,RAIZ4.SA,2.000000,Vender
2,PETR4.SA,27.940846,Manter


In [16]:
#transformed de column ticker in a list
ticker = wallet_antonio['ticker'].to_list()

In [17]:
#get the tickers price from the list wallet_antonio
from datetime import datetime
start_data = datetime.now().strftime('%Y-%m-%d')
price_quote = yf.download(ticker, start = start_data, interval="1d") #start = start_data, end = end_data)
price_quote

[*********************100%%**********************]  24 of 24 completed


Adj Close                                                   \
            AMER3.SA AZUL4.SA   BBAS3.SA BBDC3.SA   BBSE3.SA BHIA3.SA   
Date                                                                    
2023-10-04      0.75    13.25  46.799999     12.6  32.200001     0.58   

                                                ...   Volume           \
           BRKM5.SA BRSR6.SA COGN3.SA CPLE6.SA  ... LEVE3.SA MGLU3.SA   
Date                                            ...                     
2023-10-04     19.6    11.93      2.6      8.4  ...        0        0   

                                                                             \
           MRVE3.SA NEOE3.SA OIBR3.SA PETR4.SA RAIZ4.SA RANI3F.SA VALE3F.SA   
Date                                                                          
2023-10-04        0        0        0        0        0         0         0   

                     
           VBBR3.SA  
Date                 
2023-10-04        0  

[1 rows x 144 columns]

In [18]:
price_quote = price_quote['Adj Close']
price_quote = price_quote.reset_index()
price_quote.drop('Date', axis=1, inplace=True)
price_quote = price_quote.T
price_quote = price_quote.reset_index()
price_quote.columns = ['ticker','preco']
price_quote = price_quote.merge(wallet_antonio, how='inner', on='ticker')

In [19]:
status = []
for index, row in price_quote.iterrows():
    if row['preco'] > row['pm']:
      status.append("Vender")
    else:
      status.append("Manter")

In [23]:
price_quote['status'] = status
price_quote.head(3)

,ticker,preco,pm,plano,status
0,AMER3.SA,0.750000,0.780000,Manter,Manter
1,AZUL4.SA,13.250000,36.847143,Vender,Manter
2,BBAS3.SA,46.799999,30.760000,Manter,Vender


In [24]:
from winotify import Notification
import time

In [29]:
#price alert is sent through windows
teste = 0
while (teste < 1):
 for index, row in price_quote.iterrows():
   if((row['plano'] == 'Vender') and (row['status'] == 'Vender')):
     alerta = Notification(app_id = "Alerta de Ações",
                           title = "Oportunidade de Venda",
                           msg = "O preço atual da " + row['ticker'] +" é " + str(round(row['preco'],2)),
                           duration = "short",
                           launch = "https://br.financas.yahoo.com/portfolio/p_0/view/v2")
     alerta.show()
     time.sleep(5)
     teste = teste + 1